In [76]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

In [30]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-09-22 16:03:51--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   110MB/s    in 0.2s    

2024-09-22 16:03:51 (110 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [142]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [143]:
user_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()
filtered_users = user_counts[user_counts >= 200].index
filtered_isbns = isbn_counts[isbn_counts >= 100].index
filtered_df = df_ratings[df_ratings['user'].isin(filtered_users) & df_ratings['isbn'].isin(filtered_isbns)]

merged_df = pd.merge(filtered_df, df_books, on='isbn')

deduplicated_df = merged_df.drop_duplicates(subset=['title', 'user'])

pivoted_df = deduplicated_df.pivot(index='title', columns='user', values='rating').fillna(0)

rating_matrix = pivoted_df.values

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')

model_knn.fit(rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [144]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    book_ratings = rating_matrix[pivoted_df.index == book]

    if len(book_ratings) == 0:
        return [f"Book '{book}' not found.", []]

    distances, indices = model_knn.kneighbors(book_ratings, n_neighbors=6)

    recommended_books = []

    for distance, indice in zip(distances[0], indices[0]):
        if distance != 0:
            recommended_book = pivoted_df.index[indice]
            recommended_books.append([recommended_book, distance])

    recommended_books = recommended_books[::-1]

    return [book, recommended_books]

In [145]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
